
## Generalized Inverse

We show how to use EZKL to prove that we know matrices $A$ and its generalized inverse $B$. Since these are large we deal with the KZG commitments, with $a$ the polycommit of $A$, $b$ the polycommit of $B$, and $ABA = A$.


In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch

In [ ]:
class GeneralizedInverseProof(nn.Module):
    def __init__(self):
        super(GeneralizedInverseProof, self).__init__()
        self.relu = nn.ReLU()

    def forward(self,A,B):
        # some expression of tolerance to error in the inference
        return torch.sum(torch.abs(A@B@A - A)) < 0.1

circuit = GeneralizedInverseProof()

In [ ]:
gip_run_args = ezkl.PyRunArgs()
gip_run_args.input_visibility = "polycommit"  # matrix and generalized inverse commitments
gip_run_args.output_visibility = "fixed"   # no parameters used
gip_run_args.param_visibility = "fixed" # should be Tensor(True)

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
# After training, export to onnx (network.onnx) and create a data file (input.json)
shape = [10, 10]

A = 0.1*torch.rand(1,*shape, requires_grad=True)
B = A.inverse()

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      (A,B),                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input1', 'input2'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input1' : {0 : 'batch_size'},
                                    'input2' : {0 : 'batch_size'},
                                    'output' : {0 : 'batch_size'}})

d0 = ((A).detach().numpy()).reshape([-1]).tolist()
d1 = ((B).detach().numpy()).reshape([-1]).tolist()

data = dict(
    input_data=[d0, d1],
)

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [ ]:
circuit.forward(A,B)

In [ ]:

res = ezkl.gen_settings(model_path, settings_path, py_run_args=gip_run_args)

assert res == True

In [ ]:
cal_path = os.path.join("calibration.json")

data_array = (0.1*torch.rand(20,*shape).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True


In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:

# we pass the witness file to the setup function so as to prepopulate the "fixed" columns of the circuit. 
# in this case we want to force the output to be 0 meaning that the difference between the two matrices is 0
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
        witness_path = witness_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")